In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [33]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV, RidgeCV
from yellowbrick.regressor import AlphaSelection
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, StackingRegressor

In [15]:
import pandas as pd
train_set = pd.read_csv("../input/house-price-prediction-challenge/train.csv")
train_set.head(10)

In [16]:
train_set.ADDRESS.value_counts()

In [17]:
import pandas as pd
train_set = pd.read_csv("../input/house-price-prediction-challenge/train.csv")
train_set.info()


In [18]:
train_set.shape

In [19]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20 ,5))
train_set.ADDRESS = train_set.ADDRESS.apply(lambda x: x.split(',')[-1])
train_set.ADDRESS.value_counts()[:20].plot(kind='bar')
plt.show()


In [20]:
plt.figure(figsize=(20 ,8))
train_set.groupby(['ADDRESS']).sum().UNDER_CONSTRUCTION.nlargest(10).plot(kind='bar')
plt.show()

In [21]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20 ,20))
train_set.groupby(['ADDRESS']).sum().SQUARE_FT.nlargest(10).plot(kind='pie')
plt.show()

In [22]:
plt.figure(figsize=(20 ,8))
train_set.groupby(['ADDRESS']).sum().UNDER_CONSTRUCTION.nlargest(10).plot(kind='bar')
plt.show()

In [23]:
top_cities = train_set.ADDRESS.value_counts().nlargest(15).index
train_set.ADDRESS = train_set.ADDRESS.apply(lambda x: x if x in top_cities else 'Other')
train_set.ADDRESS.value_counts()

In [24]:
def label_encoder(train_set, column_name):
    label_encoder = preprocessing.LabelEncoder()

    train_set[column_name]= label_encoder.fit_transform(train_set[column_name])
    print(column_name)
    for i in range(len(train_set[column_name].unique())):
        print("For {} : {}".format(i, label_encoder.inverse_transform([i])))
    print('-'*10)
    print(train_set[column_name].value_counts())
    print('-'*10)
    
    return train_set[column_name], label_encoder

In [25]:
from sklearn import preprocessing

train_set['POSTED_BY'], label_encoder_posted_by = label_encoder(train_set, 'POSTED_BY')

train_set

In [26]:

    train_set['ADDRESS'], label_encoder_posted_by = label_encoder(train_set, 'ADDRESS')
train_set


In [27]:

train_set['BHK_OR_RK'], label_encoder_posted_by = label_encoder(train_set, 'BHK_OR_RK')
train_set

In [28]:
train_set['BHK_NO.'].value_counts()

In [29]:
import numpy as np
train_set=train_set.astype("TARGET(PRICE_IN_LACS)": int64)
train_set.head(10)


In [36]:
X = train_set.iloc[:, :-1]
y = train_set.iloc[:, -1]

In [37]:
scaler = StandardScaler()
norm = Normalizer()

X_stand = scaler.fit_transform(X)
X_norm = norm.fit_transform(X)


In [39]:
X_stand_train, X_stand_test, y_stand_train, y_stand_test = train_test_split(X_stand, y, test_size=0.33, random_state=42)
X_norm_train, X_norm_test, y_norm_train, y_norm_test = train_test_split(X_norm, y, test_size=0.33, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

MODELING

In [ ]:
estimators = [('Gradient', Gradient),('svr', AdaBoost)]
reg = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
reg.fit(X_norm_train, y_norm_train)
y_norm_pred = reg.predict(X_norm_test)

In [40]:
def make_model(name, model):
    clf = model.fit(X_train, y_train)
    clf_stand = model.fit(X_stand_train, y_stand_train)
    clf_norm = model.fit(X_norm_train, y_norm_train)
    y_pred = clf.predict(X_test)
    y_stand_pred = clf_stand.predict(X_stand_test)
    y_norm_pred = clf_norm.predict(X_norm_test)
    print(name)
    print("For orignal:")
    print("R2 : ",r2_score(y_test, y_pred))
    print("MSE : ",mean_squared_error(y_test, y_pred))
    print("-"*10)
    print("For stand:")
    print("R2 : ",r2_score(y_stand_test, y_stand_pred))
    print("MSE : ",mean_squared_error(y_stand_test, y_stand_pred))
    print("-"*10)
    print("For normalized:")
    print("R2 : ",r2_score(y_norm_test,y_norm_pred))
    print("MSE : ",mean_squared_error(y_norm_test, y_norm_pred))
    print("="*10)
    return clf, clf_stand, clf_norm
_,_, Linear = make_model("Linear Regression", LinearRegression())
_,_, Lasso = make_model("Lasso Regression", LassoCV(cv=10))
_,_, Ridge = make_model("Ridge Regression", RidgeCV((0.1, 1.0, 5.0, 10)))
_,_, ElasticNet = make_model("Elastic Net", ElasticNetCV(l1_ratio=0.5))
_,_, ElasticNet = make_model("Elastic Net", ElasticNetCV())
_,_, Gradient = make_model("Gradient", GradientBoostingRegressor())
_,_, AdaBoost = make_model("Ada", AdaBoostRegressor())

In [41]:
estimators = [('Gradient', Gradient),('svr', AdaBoost)]
reg = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
reg.fit(X_norm_train, y_norm_train)
y_norm_pred = reg.predict(X_norm_test)

In [42]:
print("For normalized:")
print("R2 : ",r2_score(y_norm_test,y_norm_pred))
print("MSE : ",mean_squared_error(y_norm_test, y_norm_pred))
print("="*10)